In [1]:
"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv'] 
"""
import pandas as pd
import re, os 
import datetime
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
from fyenn_class import pd_loaddata
warnings.filterwarnings("ignore")
# os.listdir('./data_down')
pd.set_option("display.max_rows", 15, "display.max_columns", None, 'display.float_format', lambda x: '%.3f' % x)
# pd.set_option()


In [158]:
# read data
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
# df_esti = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
# df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
# df_bill = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
# df_bcaw = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv') 
# df_bill_type = pd.read_csv('./data_down/' + 'dim_dsc_billing_type_info.csv', sep = '\001') 
"""
pic part data
"""
sap_fina = pd.read_csv('./data_down/' + 'smart_hr_pl_0l.csv', sep = '\001') 
sap_fina['mm'] = [str(i).zfill(2) for i in sap_fina['mm']]
sap_fina['yrmon'] = sap_fina[['yy', 'mm']].astype(str).apply(lambda x: ''.join(x), axis = 1)
# levin_table = pd.read_excel('./data_down/' + 'OU_Segementation2.xlsx', sheet_name = 2) 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+\.{1}', '', i) for i in list(df.columns)]

for i in [df_oppa, ]: #df_bill, df_bill_type, df_esti,  df_cost
    try:
        colname_modi(i)
    except:
        pass
del i
# rel = rel.drop(['SAP Customer', 'Finance Remark'], axis=1)

rel = rel[~rel['SAP Cost Center'].str.match('中|国')] 
rel.columns = ['cc_in_ou', 'sap_cust', 'bca', 'account_name_en']
rel['five_cc'] = rel['cc_in_ou'].str.slice(0,5)
rel2 = rel[['cc_in_ou', 'sap_cust', 'bca', 'five_cc']].merge(
    rel[['account_name_en', 'five_cc']].query("account_name_en != '(blank)'"
    ), on = 'five_cc', how = 'left').drop_duplicates()
rel2 = rel2.sort_values(['account_name_en', 'bca']).drop_duplicates(subset = 'cc_in_ou')

cc = pd_loaddata.pd_excel('./data_down/DSC成本中心_Complete', 0)
cc = cc.iloc[1:, [4,12]]
cc.columns = ['bg', 'cc_mapper']
cc['bg'] = cc['bg'].str.extract('(\w+$)')


# rel = pd_loaddata.pd_excel('./data_down/BCA与CC对照表_20211209', 1)
# rel.columns = list(rel.iloc[0])
# rel = rel.iloc[1:,]


csv_read


In [3]:
"""
my functions.
to customer level. functions set up.
"""
def data_clean(df, substr_year, **kwargs):
    """
    where u can set for month/year level.
    """
    df = df[~df['account_name_en'].isna()]
    df[substr_year] = df[substr_year].astype(str).str.slice(0,4) 
    df = df.groupby(['account_name_en', 'sap_cust', substr_year]).agg({
        **kwargs
    })
    df = df.rename({substr_year: 'month'}, axis = 1)
    return df 

def to_cust_level(df,  substr_year, **kwargs):
    """
    (df,  substr_year, **kwargs) 必须要有ou_code or cost_center!! 
    merge all to cust level \
        to_cust_level(df,  substr_year, **kwargs):
    """
    try:
        df1 = df.merge(rel2, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df1 = df.merge(rel2, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
    return data_clean(df1, substr_year,  **kwargs)

    
"""
opportunity select cols.
"""

def oppo_mas(op):
    """
    opportunity select cols.
    """
    columns = [
        'opportunity_number', 'account_name_en', 'account_name_cn', 
        'actual_close_date', 'createdon', 'expected_golive_date', 
        'annual_average_gross_profit','annual_average_revenue', 'bca_ref_number', 'bg', 'commission', 
        'contract_end_date', 'contract_term', 
        'milestone', 'opportunity_type', 'product1', 'product2','product3','product4', 'sector', 'territory',
        'contract_value', 'salesperson1', 'salesperson2', 'salesperson3', 'salesperson4'
        ]
    op = op[columns]
    
    def mon_mani(op, col):
        """钱多点"""
        op = op[op[col].astype(str).str.match('(\d+\.)').fillna(False)]
        op[col] = op[col].astype(float) * 1000    
        return op  
    for i in ['contract_value', 'annual_average_revenue', 'annual_average_gross_profit']:
        op = mon_mani(op, i)
    
    def op_cals(sap):
        """
        对op 内的财务数据进行sap相似处理.
        """
        sap['cogs'] = sap['annual_average_revenue'] - sap['annual_average_gross_profit']
        sap['gp_margin'] = sap['annual_average_gross_profit'] / sap['annual_average_revenue'] 
        sap['gp_margin'].replace([-np.inf, np.inf],  0, inplace = True)
        return sap
    op = op_cals(op)

    def time_mani(op, col):
        """
        时间格式化.
        """
        op[col] = op[col].str.slice(0,10).fillna(pd.NaT) 
        op[col] = pd.to_datetime(op[col])
        return op

    for i in ['actual_close_date', 'createdon', 'expected_golive_date', 'contract_end_date']:
        op = time_mani(op, i)

    
    op = op[~op['account_name_en'].str.contains('Freight Forwarding ').fillna(False)]
    op = op[op['account_name_en'] != 'Freight Forwarding Customer']
    # 删除同一批次导入的数据. 第一次.
    op['time_var'] = op['actual_close_date'] - op['createdon'] 
    op['time_var'] = op['time_var'].astype(str).str.extract('(\d+)').fillna(0).astype(int)

    return op
    

In [ ]:
# """
# ~billing~
# bill detail table now having 50+ bms_types. which is unacceptalble/
# """
# def data_load_bms():
#     df = df_bill.copy()
#     df['month'] = df['bms_bill_end_date'].astype(str).str.slice(0,6) 
#     df = df.merge(df_bill_type, left_on = 'bms_fee_type', right_on = 'billing_fee_type_code', how = 'left')
#     bms = df.pivot_table(index= ['cost_center', 'month'],\
#         columns= 'category',
#         values='bms_amount',).reset_index().fillna(0)
#     return bms
# bms = data_load_bms()
# bms['yr'] = bms['month'].str.slice(0,4)
# bms_cus_yr = to_cust_level(df = bms, substr_year = 'month' ,\
#     cost_center = set, 
#     人力 = 'sum',
#     场地 = 'sum',
#     增值 = 'sum',
#     报关代理 = 'sum',
#     操作 = 'sum',
#     短驳运输 = 'sum',
#     索赔及返利 = 'sum',
#     设备使用 = 'sum' ).reset_index()
# bms_cus_yr.head(2)

### finish load
---


> some opportunity analysis

In [ ]:
# op = oppo_mas(df_oppa)
# op['mont_code'] = op['expected_golive_date'].astype(str).str.slice(0,7).str.replace('-', '')
# # op.head(2)
# op['prodct'] = op[['product1','product2', 'product3', 'product4']].astype(str).apply(lambda x: ','.join(x), axis = 1)
# op['prodct'] = op['prodct'].str.replace(',nan', '')
# op = op.drop(['product1','product2', 'product3', 'product4'], axis = 1)
# op['commission'] = op['commission'].replace('TRUE', 1).replace('FALSE', 0)
# # # from sklearn.preprocessing import MinMaxScaler
# # # scaler = MinMaxScaler() 
# # plt_cs = op[op['milestone'] == 'Contract Signed']
# # def bubble_plt(plot_data, color):
# #     fig = px.scatter(
# #         data_frame= plot_data, x = 'annual_average_revenue', y= 'time_var', hover_data=['account_name_en'],
# #         size = 'annual_average_gross_profit', color = color, title='  ')  
# #     # fig.add_vline(x = vline, line_width=3, line_dash="dash", line_color="black", opacity = .4)
# #     return fig
# # # bubble_plt(plt_cs,   'sector')
# # op2

---
# opdata
"""manipulation"""

In [4]:
import datetime
import calendar
from tqdm import tqdm
op = oppo_mas(df_oppa)
def op_more(op):
    op['mont_code'] = op['expected_golive_date'].astype(str).str.slice(0,7).str.replace('-', '')
    # op.head(2)
    op['prodct'] = op[['product1','product2', 'product3', 'product4']].astype(str).apply(lambda x: ','.join(x), axis = 1)
    op['sales'] = op[['salesperson1','salesperson2', 'salesperson3', 'salesperson4']].astype(str).apply(lambda x: ','.join(x), axis = 1)
    op['prodct'] = op['prodct'].str.replace(',nan', '')
    op['sales'] = op['sales'].str.replace(',nan', '')

    op = op.drop(['product1','product2', 'product3', 'product4'], axis = 1)
    op = op.drop(['salesperson1','salesperson2', 'salesperson3', 'salesperson4'], axis = 1)

    op['commission'] = op['commission'].replace('TRUE', 1).replace('FALSE', 0)
    return op
op = op_more(op).merge(rel[['account_name_en', 'cc_in_ou']], on = 'account_name_en', how = 'inner')


In [5]:
op2 = op[op['milestone'] == 'Contract Signed']
"""
以月份为组的sum? 这里合理吗?
"""
op2 = op2.groupby(['account_name_en', 'account_name_cn', 'bg', 'mont_code', 'opportunity_number', 'cc_in_ou']).agg({
    'commission':sum,
    'annual_average_gross_profit' :sum, 
    'annual_average_revenue' :sum, 
    'contract_term' :sum, 
    'contract_value' :sum, 
    'prodct': set,
    'sales':set
}).reset_index()
op2['mon_to_yrend'] = (op2['mont_code'].str.slice(0,4) + '12').astype(int) - op2['mont_code'].astype(int)
monlist = [
    'annual_average_gross_profit',
    'annual_average_revenue',
    # 'contract_term',
    'contract_value',] 

op2['mont_code'] = op2['mont_code']+'01'


def add_months(sourcedate, months):
    """
    make duplicates
    """
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day).strftime('%Y%m')
# add_months(pd.to_datetime(op2['mont_code'])[0] ,1) 
q = []
for i, j in zip(pd.to_datetime(op2['mont_code']), op2['contract_term'].astype(int)):
    q.append( add_months(i, j))
op2['end_mont_code'] = q
op2['mon_to_yrend_future'] = (
    op2['end_mont_code'].astype(str).str.slice(0,4) + '12').astype(int) - op2['end_mont_code'].astype(int)


In [145]:
# df_oppa.sort_values('account_name_en')

In [6]:
op2['dupli'] = op2['end_mont_code'].str.slice(0,4).astype(int) - op2['mont_code'].str.slice(0,4).astype(int)
tst = pd.DataFrame()
for i, j in tqdm(zip(range(op2.shape[0]), op2['dupli'])):
    """
    复制第一条数据, 产生每年年化收益.
    """
    try:
        tst = tst.append(pd.concat([pd.DataFrame(op2.iloc[i, :]).T] * int(j)))
    except:
        pass


for i in tqdm(monlist):
    """
    第一条数据已经转化为实际年收入
    """
    name = i
    op2[name] = op2[i].astype(float) / 12 * op2['mon_to_yrend']

3517it [00:07, 448.88it/s]
100%|██████████| 3/3 [00:00<00:00, 600.01it/s]


In [7]:
"""
除掉第一年数据.
"""
tst['row_num'] = tst.groupby(['account_name_en', 'mont_code','annual_average_gross_profit']).cumcount() + 1
tst['mont_code'] = tst['mont_code'].str.slice(0,4).astype(int) + tst['row_num'].astype(int)
op2['mont_code'] = op2['mont_code'].str.slice(0,4).astype(int)
"""
中间年
"""
mid_yrs = tst.groupby([
    'account_name_en', 'contract_term', 'annual_average_gross_profit'], as_index=False
    ).apply(lambda x: x.iloc[:-1])

"""
结束年
"""
end_yrs = tst.groupby([
    'account_name_en', 'contract_term', 'annual_average_gross_profit'], as_index=False
    ).apply(lambda x: x.iloc[-1])

for i in tqdm(monlist):
    """
    结束年数据已经转化为实际年收入
    """
    name = i
    end_yrs[name] = end_yrs[i].astype(float) / 12 * (12 - end_yrs['mon_to_yrend'])

100%|██████████| 3/3 [00:00<00:00, 1501.00it/s]


In [8]:
"""
concat all yrs to one
"""
op_all = pd.concat([op2, mid_yrs, end_yrs]).sort_values(
    ['account_name_en', 'mont_code']
    ).reset_index(

    ).drop_duplicates(subset = monlist + ['mont_code'])
op_all = op_all.drop('index', axis = 1)

In [427]:
# duplicated might caused by the rows who are first and last at the same time.? 
# not sure about it , but never mind since the value of these amount is short.
# sap.head(4)


In [9]:
op_all['prodct'] =[' '.join(i) for i in op_all['prodct']]
op_all['sales'] =[' '.join(i) for i in op_all['sales']]
"""
年粒度
"""
op_all = op_all.groupby(['account_name_en', 'account_name_cn', 'bg', 'mont_code', 'cc_in_ou']).agg(
    {
    'commission':sum,
    'annual_average_gross_profit' :sum, 
    'annual_average_revenue' :sum, 
    'contract_term' :'mean', 
    'contract_value' :sum, 
    'prodct': set  ,
    'sales': set,
    'opportunity_number': [set, 'count'],
    }
).reset_index()
op_all[('prodct', 'set')] = [';'.join(i) for i in op_all[('prodct', 'set')]]
op_all[('sales', 'set')] = [';'.join(i) for i in op_all[('sales', 'set')]]

In [10]:
op_all.columns = op_all.columns.droplevel(1)
list = op_all.columns.values
list[-1] = 'opportunity_number_num'
op_all.columns = list
del list
# del df_bill, df_bill_type, df_esti, i, j
# del q, op, op2, tst, mid_yrs, end_yrs
# del tst2

`===========================`
<br>
`merge sap`

In [11]:
"""
~sap~
"""
def load_data_sap():
    df = sap_fina.copy()
    df = df.rename({'prctr': 'cost_center'}, axis =1 )
    return df

sap_fina = load_data_sap()

# sap = to_cust_level(sap_fina, 'yrmon', \
#     cost_center = set, 
#     income_amt = sum,
#     gross_profit_amt = sum,
#     human_cost_amt = sum, 
#     service_outsourcing_amt = sum,
#     labor_service_outsourcing_amt = sum
 
#     ).reset_index()
    
# sap = sap.rename({'yrmon':'mont_code'}, axis =1 )
# sap['lb_cost'] = sap.iloc[:, -3:].sum(axis = 1)
# sap.head(3)
sap_fina = sap_fina.rename({'yrmon':'mont_code'}, axis =1 )
sap = sap_fina
sap['cc_mapper'] = sap['cost_center'].str.slice(0,10)
sap = sap.merge(cc, on = 'cc_mapper', how = 'inner') 
sap['bg'] = sap['bg'].replace('香港', 'HK')
sap = sap.merge(rel2.drop(['bca', 'five_cc'], axis = 1).drop_duplicates(), left_on='cost_center', right_on='cc_in_ou', how = 'inner')

In [12]:
"""
多行op 匹配单行的sap  
时间节点要用 actual go live 的yr. 对应当月的sap财务状况.
cc level modified
"""
sap['mont_code'] = sap['mont_code'].astype(int)
# sap['cost_center'] = [','.join(i) for i in sap['cost_center']]

sap = sap.groupby(['account_name_en', 'cost_center', 'sap_cust', 'yy', 'bg']).agg(
    {
        # 'cost_center': set, 
        'income_amt': sum,
        'gross_profit_amt': sum, 
        'human_cost_amt': sum, 
        'service_outsourcing_amt': sum,
        'labor_service_outsourcing_amt': sum, 
        # 'lb_cost': sum
    }
).reset_index()

# sap['cost_center'] = [','.join(i) for i in sap['cost_center']]
sap['mont_code'] = sap['yy']

In [13]:
op_all= op_all.rename({'cc_in_ou':'cost_center'}, axis =1 )

In [14]:
sap.head(2)

,account_name_en,cost_center,sap_cust,yy,bg,income_amt,gross_profit_amt,human_cost_amt,service_outsourcing_amt,labor_service_outsourcing_amt,mont_code
0,"APEX Cargo Service Co., Ltd (ACS)",EIXCNBGGFF,Freight Forwarding EI,2021,EI,211150379.530,755858.640,218740.380,609.620,0.000,2021
1,APPLE,APPLECDSBS,APPLE,2019,TD,-3396.220,-138707.410,694.010,341.880,0.000,2019


In [30]:
# op_all.sort_values('account_name_en').head(22)

In [26]:
# sap[sap['account_name_en'] .str.contains('Abb').fillna(False)]

In [32]:
# op_all[op_all['account_name_en'].str.contains('Abb').fillna(False)]

In [432]:
###
op_all

,account_name_en,account_name_cn,bg,mont_code,cost_center,commission,annual_average_gross_profit,annual_average_revenue,contract_term,contract_value,prodct,sales,opportunity_number,opportunity_number_num
0,"APEX Cargo Service Co., Ltd (ACS)",上海丹正国际货运代理有限公司,EI,2021,EIXCNBGGFF,1,147060.000,13853830.000,12.000,13853830.000,Freight Forwarding,Bo Zhou (SF-DHL CN),{Z009688},1
1,"APEX Cargo Service Co., Ltd (ACS)",上海丹正国际货运代理有限公司,EI,2022,EIXCNBGGFF,1,147060.000,13853830.000,12.000,13853830.000,Freight Forwarding,Bo Zhou (SF-DHL CN),{Z009688},1
2,APPLE,苹果,CM,2018,APPLESHBSH,1,0.000,0.000,36.000,0.000,Warehousing,Vivien He (SF-DHL CN),{Z005265},1
3,APPLE,苹果,CM,2019,APPLESHBSH,1,307930.000,2400000.000,36.000,7200000.000,Warehousing,Vivien He (SF-DHL CN),{Z005265},1
4,APPLE,苹果,CM,2020,APPLESHBSH,1,307930.000,2400000.000,36.000,7200000.000,Warehousing,Vivien He (SF-DHL CN),{Z005265},1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3351,桂林深科技有限公司,桂林深科技有限公司,TSC,2021,GLSKJDGWHS,1,26241.667,97115.000,3.000,45673.333,Warehousing,"潘荣,Xiaogang Yang (SF-DHL CN);潘荣,谢姝贤","{Z010078, Z009647}",2
3352,桂林深科技有限公司,桂林深科技有限公司,TSC,2022,GLSKJDGWHS,0,40791.667,134241.667,3.000,140533.333,Warehousing,"潘荣,Xiaogang Yang (SF-DHL CN)",{Z010078},1
3353,翰林汇信息产业股份有限公司,翰林汇信息产业股份有限公司,CR,2021,NIYAZSHSJG,0,10416.667,133750.000,12.000,133750.000,Other service,Hawkins He2 (SF-DHL CN),{Z009910},1
3354,翰林汇信息产业股份有限公司,翰林汇信息产业股份有限公司,CR,2022,NIYAZSHSJG,0,14583.333,187250.000,12.000,187250.000,Other service,Hawkins He2 (SF-DHL CN),{Z009910},1


In [429]:

# merge prepared
op_sap = op_all.merge(
    sap, on = ['account_name_en', 'mont_code', 'bg', 'cost_center'], how = 'right'
    ).sort_values(['account_name_en','cost_center', 'mont_code']);op_sap.shape



(1838, 21)

In [431]:
op_sap.to_csv('./data_up/opp_all_sap.csv', index = None, encoding = 'utf_8_sig')

In [83]:
# op_all.to_csv('./data_up/op_all.csv', index = False, encoding = 'utf_8_sig')
# sap.to_csv('./data_up/sap.csv', index = False, encoding = 'utf_8_sig')
# op_sap_bg_summary.to_csv('./data_up/op_sap_bg_sum.csv', index = False, encoding = 'utf_8_sig')

In [50]:
op_sap = op_sap.groupby(['account_name_en','account_name_cn','bg', 'mont_code', 'sap_cust'], as_index=False).agg(
    {   
        'commission':'mean',
        'annual_average_gross_profit' :'mean', 
        'annual_average_revenue' :'mean', 
        'contract_term' :'mean', 
        'contract_value' :'mean', 
        'prodct': 'first'  ,
        'sales': 'first',
        # 'opportunity_number': [set, 'count'],
        'income_amt': sum,
        'gross_profit_amt': sum, 
        'human_cost_amt': sum, 
        'service_outsourcing_amt': sum,
        'labor_service_outsourcing_amt': sum, 
    }
)

In [51]:
op_sap = op_sap[~op_sap['annual_average_gross_profit'].isna()]
op_sap['gp_est_over_real'] = op_sap['annual_average_gross_profit'] / op_sap['gross_profit_amt']
op_sap['rev_est_over_real'] = op_sap['annual_average_revenue'] / op_sap['income_amt']
op_sap = op_sap[~op_sap['prodct'].str.contains('Freight Forwarding').fillna(False)]
monlist_all = monlist + ['income_amt', 'gross_profit_amt']

In [43]:
# op[op['cc_in_ou'] == 'AMATXBJKCS']

In [54]:
# op_sap

In [126]:
# op_sap.to_pickle('./data_up/op_sap.p')
# op_sap.head(4)
# op_sap['account_name_en'].nunique()


336

In [55]:
op_sap_bg_summary = op_sap.groupby(['bg', 'mont_code'], as_index = False)[monlist_all].sum()
op_sap_bg_summary['gp_est_over_real'] = op_sap_bg_summary['annual_average_gross_profit'] / op_sap_bg_summary['gross_profit_amt']
op_sap_bg_summary['rev_est_over_real'] = op_sap_bg_summary['annual_average_revenue'] / op_sap_bg_summary['income_amt']

In [65]:
op_sap_bg_summary_21 = op_sap_bg_summary.query("mont_code == 2021 & bg != 'SPD' & bg != 'ZQ'")
plt = op_sap_bg_summary_21

In [76]:

# plt =op_sap[op_sap['mont_code'] < 202106]
# plt[~plt['annual_average_gross_profit'].isna()]
fig = go.Figure()

# fig.add_trace( go.Bar(x = plt['bg'], y = plt['annual_average_revenue'],\
#     text = (plt['income_amt']/100000000).round(2).astype(str) + '亿\t' +plt['bg'], name = 'opportunity_rev'))
# fig.add_trace( go.Bar(x = plt['bg'], y = plt['income_amt'],\
#     text = (plt['income_amt']/100000000).round(2).astype(str) + '亿\t' +plt['bg'], name = 'sap_income_amt'))


# fig.add_trace(go.Bar(x = plt['bg'], y = plt['annual_average_gross_profit'], \
#     text = (plt['annual_average_gross_profit'].astype(int)/100000000).round(2).astype(str) + '亿\t' +plt['bg'], name = 'opportunity_gp'))
 
# fig.add_trace(go.Bar(x = plt['bg'], y = plt['gross_profit_amt'], \
#     text = (plt['gross_profit_amt']/100000000).round(2).astype(str) + '\t' +plt['bg'], name = 'sap_gross_profit_amt'))
 
fig.add_trace(go.Bar(x = plt['bg'], y = plt['gp_est_over_real'], \
    text = plt['gp_est_over_real'].round(2).astype(str) + '\t' +plt['bg'], name = 'gp_est_over_real'))
fig.add_trace( go.Bar(x = plt['bg'], y = plt['rev_est_over_real'],\
    text = plt['rev_est_over_real'].round(2).astype(str) + '\t' +plt['bg'], name = 'rev_est_over_real'))
# fig.write_html('./data_up/bg_proportion.html')

In [93]:
# op 全部

In [87]:
op3 = op[op['milestone'] != 'Contract Signed']
op3 = op3[~op3['milestone'].str.contains('(Close)')]
op3 = op3[op3['expected_golive_date'] > '2021-12-31']
op3 = op3[op3['expected_golive_date'] < '2022-06-30']
# op3 = op3.groupby('bg')[monlist].sum().reset_index()
# op22 = op_all[op_all['mont_code'] == 2022].groupby(['bg', 'mont_code'])[monlist].sum().reset_index()
# op3_22 = pd.concat([op3 , op22], axis = 0).drop('mont_code', axis = 1).groupby('bg', as_index = False).sum()

In [95]:
# opened
op3_old = op3[ op3['account_name_en'].isin(op_all['account_name_en'])]
op3_new = op3[~op3['account_name_en'].isin(op_all['account_name_en'])]


In [107]:
df_oppa['milestone'].unique()

array(['Potential Opportunity', 'Closed-Canceled', 'Closed-Lost',
       'Contract Signed', 'Early Lead', nan, 'Qualified', 'Shortlisted',
       'Opportunity Gain', 'Peer Review', 'DCI Conducted'], dtype=object)

In [103]:
df_oppa['opportunity_type'].unique()

array(['Renewal', 'New from Existing', 'New from New', nan], dtype=object)

In [100]:
op['opportunity_type'].unique()

array(['Renewal', 'New from Existing', 'New from New'], dtype=object)

In [167]:
# opened
op_20 = df_oppa[df_oppa['expected_golive_date']<'2020-12-31']
op_20_old = op_20[ op_20['account_name_en'].isin(op_20[op_20['milestone'].isin([ 
       'Contract Signed'])]['account_name_en'])]
# op_new = op_20[~op_20['account_name_en'].isin(op_20['account_name_en'])]
op_20_new = op_20[ op_20['account_name_en'].isin(op_20[~op_20['milestone'].isin([ 
       'Contract Signed'])]['account_name_en'])]

# 491 1171


In [373]:
def all_op_by_time(condition1, condition2, close_or_open):
    """
    所有的op, 筛选时间. 分类到旧客, 新客.
    """
    op21 = df_oppa[(df_oppa['expected_golive_date']>=condition1) & (df_oppa['expected_golive_date']<=condition2)]
    op21_old = op21[op21['account_name_en'].isin(op_20_old['account_name_en'].unique())] 
    op21_new = op21[~op21['account_name_en'].isin(op_20_old['account_name_en'].unique())] 
    if close_or_open == 1:
        op21_old = op21_old[op21_old['milestone'].isin(['Contract Signed', 'Closed-Canceled', 'Closed-Lost',])]
        op21_new = op21_new[op21_new['milestone'].isin(['Contract Signed', 'Closed-Canceled', 'Closed-Lost',])]
    else: 
        op21_old = op21_old[~op21_old['milestone'].isin(['Contract Signed', 'Closed-Canceled', 'Closed-Lost',])]
        op21_new = op21_new[~op21_new['milestone'].isin(['Contract Signed', 'Closed-Canceled', 'Closed-Lost',])]
    op21_old['annual_average_gross_profit'] = op21_old['annual_average_gross_profit'].astype(float) *1000
    op21_new['annual_average_gross_profit'] = op21_new['annual_average_gross_profit'].astype(float) *1000
    op21_old['annual_average_revenue'] = op21_old['annual_average_revenue'].astype(float) *1000
    op21_new['annual_average_revenue'] = op21_new['annual_average_revenue'].astype(float) *1000
    return op21_old , op21_new

op21_old = all_op_by_time('2020-12-31', '2021-12-31', 1)[0]
op21_new = all_op_by_time('2020-12-31', '2021-12-31', 1)[1]
    # op21_new['milestone'].value_counts()
    # 182/(253 +182+132)
    # op21_old['milestone'] .value_counts()
    # 266/ (266 +93+152)
    # op21_new

def conversion_rate(col, name):
    name = name + '_conversion_rate'
    op21_old_bg = pd.DataFrame(op21_old.groupby(['bg', 'milestone', 'opportunity_type'])[col].sum())
    op21_new_bg = pd.DataFrame(op21_new.groupby(['bg', 'milestone', 'opportunity_type'])[col].sum())
    op21_old_bg.columns = [name]
    op21_old_bg = op21_old_bg.reset_index()
    op21_new_bg.columns = [name]
    op21_new_bg = op21_new_bg.reset_index()
    op21_old_bg['tt'] = op21_old_bg.groupby('bg')[name].transform( 'sum')
    op21_new_bg['tt'] = op21_new_bg.groupby('bg')[name].transform( 'sum')
    op21_old_bg[name] = op21_old_bg[name] / op21_old_bg['tt']
    op21_new_bg[name] = op21_new_bg[name] / op21_new_bg['tt']
    return op21_old_bg, op21_new_bg

In [400]:
# op21_old = op21_old.merge(op21_old_bg[op21_old_bg['milestone'] == 'Contract Signed'][['bg', 'frac', 'milestone']], on = ['bg', 'milestone'])
# op21_new = op21_new.merge(op21_new_bg[op21_new_bg['milestone'] == 'Contract Signed'][['bg', 'frac', 'milestone']], on = ['bg', 'milestone'])
# op21_new = op21_new[op21_new['opportunity_record'] != 'Straightforward Renewals']
# op21[~op21['milestone'].isin(['Contract Signed', 'Closed-Canceled', 'Closed-Lost',])]


In [407]:

op21_old_bg['type'] = '老客'
op21_new_bg['type'] = '新客'

pd.concat([op21_old_bg, op21_new_bg]).drop('index', axis = 1)\
    # .to_csv('./data_up/cvrt_rate_21.csv', index= None, encoding = 'utf_8_sig')

,bg,milestone,gp_sum,tt,conversion_rate,type
0,Auto,Closed-Canceled,11862000.000,41246400.000,0.288,老客
1,Auto,Closed-Lost,12467930.000,41246400.000,0.302,老客
2,Auto,Contract Signed,16916470.000,41246400.000,0.410,老客
3,CR,Closed-Canceled,3353000.000,31315370.000,0.107,老客
4,CR,Closed-Lost,3895520.000,31315370.000,0.124,老客
...,...,...,...,...,...,...
26,TSC,Closed-Canceled,17491000.000,98978970.000,0.177,新客
27,TSC,Closed-Lost,11732100.000,98978970.000,0.119,新客
28,TSC,Contract Signed,69755870.000,98978970.000,0.705,新客
29,ZQ,Closed-Lost,441860.000,1832030.000,0.241,新客


In [414]:
# (op21_old['annual_average_revenue'] * op21_old['frac']).sum() + (op21_new['annual_average_revenue'] * op21_new['frac']).sum()
# new_gp
# old_rev
df = conversion_rate('annual_average_gross_profit', 'rev')[0]
df['type'] = 'old'
df1 = conversion_rate('annual_average_gross_profit', 'rev')[0]
df1['type'] = 'new'
pd.concat([df, df1])\
    .to_csv('./data_up/cvrt_rate_21_gp.csv', index= None, encoding = 'utf_8_sig')

# conversion_rate('annual_average_revenue', 'rev')[1] 

In [393]:

def convert_22(col):
    op22_old = all_op_by_time('2021-12-31', '2024-12-31', 0)[0]
    op22_new = all_op_by_time('2021-12-31', '2024-12-31', 0)[1]
    old_rev = conversion_rate('annual_average_revenue', 'rev')[0]
    new_rev = conversion_rate('annual_average_revenue', 'rev')[1]
    old_gp = conversion_rate('annual_average_gross_profit', 'gp')[0]
    new_gp = conversion_rate('annual_average_gross_profit', 'gp')[1]
    if col == 1:
        col1 = 'annual_average_revenue'
        col2 = 'rev_conversion_rate'
        col3 = 'rev_cvted_22'
        old_rev = old_rev
        new_rev = new_rev
    else:
        col1 = 'annual_average_gross_profit'
        col2 = 'gp_conversion_rate'
        col3 = 'gp_cvted_22'
        old_rev = old_gp
        new_rev = new_gp
    op22_old = pd.DataFrame(op22_old.groupby(['bg', 'opportunity_type'], as_index = False)[col1].sum())
    op22_new = pd.DataFrame(op22_new.groupby(['bg', 'opportunity_type'], as_index = False)[col1].sum())
    op22_old_converted = op22_old.merge(
        old_rev.query("milestone == 'Contract Signed'").drop(['tt', 'milestone'], axis = 1))
    op22_old_converted[col3] = op22_old_converted[col1] * op22_old_converted[col2]
    op22_new_converted = op22_new.merge(
        new_rev.query("milestone == 'Contract Signed'").drop(['tt', 'milestone'], axis = 1))
    op22_new_converted[col3] = op22_new_converted[col1] * op22_new_converted[col2]
    return op22_old_converted, op22_new_converted


In [415]:
op22_old_rev_converted = convert_22(1)[0]
op22_new_rev_converted = convert_22(1)[1]
op22_old_gp_converted = convert_22(2)[0]
op22_new_gp_converted = convert_22(2)[1]

In [423]:


op22_old_gp_converted['type'] = 'old'
op22_new_gp_converted['type'] = 'new'

pd.concat([op22_old_gp_converted, op22_new_gp_converted]) #.drop('index', axis = 1)\
    .to_csv('./data_up/value_cvrtd_22_gp.csv', index= None, encoding = 'utf_8_sig')

In [ ]:
# fig = go.Figure()

# # fig.add_trace( go.Bar(x = plt['bg'], y = plt['annual_average_revenue'],\
# #     text = (plt['income_amt']/100000000).round(2).astype(str) + '亿\t' +plt['bg'], name = 'opportunity_rev'))
# # fig.add_trace( go.Bar(x = plt['bg'], y = plt['income_amt'],\
# #     text = (plt['income_amt']/100000000).round(2).astype(str) + '亿\t' +plt['bg'], name = 'sap_income_amt'))


# fig.add_trace(go.Bar(x = plt['bg'], y = plt['annual_average_gross_profit'], \
#     text = (plt['annual_average_gross_profit'].astype(int)/100000000).round(2).astype(str) + '亿\t' +plt['bg'], name = 'opportunity_gp'))
 
# fig.add_trace(go.Bar(x = plt['bg'], y = plt['gross_profit_amt'], \
#     text = (plt['gross_profit_amt']/100000000).round(2).astype(str) + '\t' +plt['bg'], name = 'sap_gross_profit_amt'))

In [ ]:
# 全部的
sap_fina[sap_fina['yy'] == 2021][['income_amt', 'gross_profit_amt']].sum()

In [ ]:
# 能mapping 到的
mapped_21_sap =  op_sap_bg_summary[op_sap_bg_summary['mont_code'] == 2021][['income_amt', 'gross_profit_amt']].sum()
# fig = go.Figure(go.Pie(values = [4329535156.610, 7849952033.350 - 4329535156.610], \
#     labels= ['能map', '不能map'], text = ['43.3亿', '35.2亿'],
#     name = '在商机与sap的mapping中的金额分布'))
# fig.update_layout(
#     margin=dict(l=5, r=5, t=5, b=5),
#     # paper_bgcolor="LightSteelBlue",a/
# )



In [ ]:
mapped_21_sap[0]*1.2 - 2952038347 - 2337495730 * .46

In [ ]:
go.Figure(go.Bar(
    y =  [2952038347.500], text = 'already had' + '\n\t' + str('29.5亿')
    )
    ).add_trace(
        go.Bar(
            y =  [2337495730.050], text = 'opened' + '\n\t' + str('23.4亿')
        )
    ).add_trace(
        go.Bar(y = [mapped_21_sap[0]*1.2 - 2952038347 - 2337495730 * .46], text = 'to go' + '\n\t' + str('11.7亿'))
        ) .update_layout(barmode='stack')

In [ ]:
op3_pred = op3_22.merge(plt[['gp_est_over_real', 'rev_est_over_real', 'bg']], on = 'bg')

In [ ]:
op3_pred['pred_annual_average_gross_profit']  = op3_pred['annual_average_gross_profit'] * op3_pred['gp_est_over_real']
op3_pred['pred_annual_average_revenue']  = op3_pred['annual_average_revenue'] * op3_pred['rev_est_over_real']

In [ ]:
op3_pred.sum()

In [ ]:
op3_22

In [ ]:
# from fbprophet import Prophet
# # df = x[['CREATION_DATE_TIME_STAMP', 'qty']]
# df_naiv = x_train 
# df_naiv = df_naiv.rename({'date1' : 'ds'}, axis = 1)
# df_naiv_test = x_test
# df_naiv_test = df_naiv_test.rename({'date1' : 'ds'}, axis = 1)
# m = Prophet()
# [m.add_regressor(i) for i in cols]

# m.fit(df_naiv)

# # from sklearn.metrics import mean_absolute_percentage_error

# # # m.predict(x_test)


# def fillna_(col, op_sap):
#     name = 'prd_' + col
#     op_sap[name] = op_sap.groupby('account_name_en')[col].cumsum()
#     op_sap[name]  = op_sap .groupby('account_name_en')[name].fillna(method = 'bfill').fillna(method = 'ffill')
#     return op_sap
# for i in ['contract_value', 'annual_average_revenue', 'annual_average_gross_profit']:
#         op_sap = fillna_(i, op_sap)

In [ ]:
# op = op[~(op['actual_close_date'] - op['createdon'] < datetime.timedelta(days = 0))] 
sap['max_mont'] = sap.groupby('account_name_en')['mont_code'].transform( 'max' )

In [ ]:
sap[sap['max_mont'] != '202110']